In [1]:
import pandas as pd
import glob
import re

In [2]:
files = glob.glob("./data/*.txt")
lines = []
for i in range(len(files)):
	with open(files[i]) as f:
		lines += f.readlines()

In [3]:
def remove_numbers(text):
	# Define regex pattern to match numbers followed by a dot and space
	pattern = r'^\d+\.\s+'
	# Replace matched pattern with an empty string
	cleaned_text = re.sub(pattern, '', text)
	cleaned_text = re.sub('"', '', cleaned_text)
	cleaned_text = re.sub('\'', "'", cleaned_text)
	return cleaned_text.strip('\n')



In [4]:
lines = [remove_numbers(line) for line in lines if line != '\n']
test_data_file = './data/lang_test_data.txt'

for line in lines:
	with open(test_data_file, 'a') as file:
		file.write(line + '*' + '\n')


In [5]:
test_data = pd.read_csv(test_data_file, delimiter='*', header=None)
test_data = test_data.drop_duplicates().reset_index()

In [166]:
import requests
import json

api_endpoint = 'http://10.10.45.1:1405/multilang-translate'
base_lang = 'english'
new_lang = 'hausa'

for i in range(len(test_data)):
	text_str = test_data[0][i]
	payload = dict(text=text_str, from_ = base_lang, to_ = new_lang)
	response = requests.post(api_endpoint, data=payload)
	if response.status_code == 200:
		response = json.loads(response)
		test_data.loc[i, 1] = response["translated"]